In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition
import numpy as np
import pickle
import tokenizer as tk

In [2]:
df = pd.read_csv(r'C:\Projects\PythonProjects\TopicModelling\Datasets\yelp-1star-complete.csv')

In [3]:
reviews = df['Review'].values
tokenized_reviews = [tk.preprocess_text(text) for text in reviews]

In [5]:
vectorizer_tf = TfidfVectorizer(max_df=0.90, min_df=50, max_features=10000)
tf_vectors = vectorizer_tf.fit_transform(tokenized_reviews)

In [8]:
lda = decomposition.LatentDirichletAllocation(n_components=3, max_iter=25, learning_method='online', random_state=42, batch_size=2000)

In [9]:
W1 = lda.fit_transform(tf_vectors)

In [10]:
H1 = lda.components_

In [12]:
num_words=20
vocab = np.array(vectorizer_tf.get_feature_names_out())
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [13]:
topics

['order pizza get food time go call place service wait drive location minute bad never take say give customer back',
 'us table wait come go service food get minute say ask drink take tell time place restaurant server one leave',
 'food chicken taste good like order place eat bad go cheese salad sauce fry get meat dry price steak rice']

In [14]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(tokenized_reviews))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [58]:
df_doc_topic

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.10,0.83,0.06,1
Doc1,0.05,0.11,0.84,2
Doc2,0.34,0.06,0.59,2
Doc3,0.08,0.35,0.57,2
Doc4,0.70,0.23,0.07,0
...,...,...,...,...
Doc404485,0.07,0.68,0.25,1
Doc404486,0.54,0.39,0.07,0
Doc404487,0.49,0.04,0.47,0
Doc404488,0.20,0.10,0.70,2
